In [2]:
import numpy as np
import matplotlib.pylab as plt
#from scipy.misc import imresize, imread
from PIL import Image
import itertools
import sklearn
from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, learning_curve, GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import keras
import glob
import os
import shutil
import random

from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tqdm import tqdm
import tensorflow.keras.backend as K
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
"""
from keras import backend as K
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, model_from_json
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D
"""

%matplotlib inline

Using TensorFlow backend.


In [ ]:
pip install --upgrade tensorflow

In [ ]:
pip install tensorflow==2.0

In [23]:
IMAGE_RESIZE = 50
BATCH_SIZE_TRAINING_LABELED_SUBSET = 16
BATCH_SIZE_TRAINING_UNLABELED_SUBSET = 8


### Organiser les images

Réorganiser les fichiers dans 0/1 dosiers et éliminer les dossers numéro patient

In [ ]:
def organize_images(data_path, pre_processed_data_path):
    files = glob.glob(data_path + '\**\*.png', recursive=True)
    class_0_path = os.path.join(pre_processed_data_path,"0")
    class_1_path = os.path.join(pre_processed_data_path,"1")
    os.makedirs(class_0_path,exist_ok=True)
    os.makedirs(class_1_path,exist_ok=True)
    
    for file in files:
        if "class0" in file:
            file_dest = os.path.join(class_0_path,os.path.basename(file))
        elif "class1" in file:  
            file_dest = os.path.join(class_1_path,os.path.basename(file))
        shutil.copyfile(file, file_dest)

organize_images("..\..\data", "..\..\pre_processed_data")

In [ ]:
xx = glob.glob('..\..\data\**\*.png', recursive=True)
print(len(xx))

In [ ]:
"""
def unique(list1): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    # print list 
    #for x in unique_list: 
    #    print x,
    return unique_list
"""

# Split dataset to Train and Test data

We will create a method that extracts 20% of the data for the test and the rest will be for the train.

In [ ]:
def extract_train_test_data(data_forlder, part_percentage):
    files_0 = glob.glob(data_forlder + '/0/*.png', recursive=True)
    files_1 = glob.glob(data_forlder + '/1/*.png', recursive=True)
    part_files_0_len = int((len(files_0)*part_percentage)/100)
    part_files_1_len = int((len(files_1)*part_percentage)/100)
    
    part_files_0_train = []
    part_files_0_test = []
    
    part_files_1_train = []
    part_files_1_test = []
    for index, file in enumerate(files_0):
        if index < part_files_0_len:
            part_files_0_test.append(os.path.abspath(file))
        else:
            part_files_0_train.append(os.path.abspath(file))
            
    for index, file in enumerate(files_1):
        if index < part_files_1_len:
            part_files_1_test.append(os.path.abspath(file))
        else:
            part_files_1_train.append(os.path.abspath(file))
    
    return part_files_0_train, part_files_1_train, part_files_0_test, part_files_1_test

In [ ]:
result = extract_train_test_data("../../pre_processed_data", 20)
part_files_0_train, part_files_1_train, part_files_0_test, part_files_1_test = result
#part_files_0, part_files_1 = resut
#print(len(part_files_0))
#print(len(part_files_1))

In [ ]:
def create_data_repos(folder, files_0, files_1):
    #create 
    os.makedirs(folder, exist_ok=True)
    
    class_0_path = os.path.join(folder,"0")
    class_1_path = os.path.join(folder,"1")
    os.makedirs(class_0_path,exist_ok=True)
    os.makedirs(class_1_path,exist_ok=True)
    
    for index, file in enumerate(files_0):
        shutil.copyfile(file, os.path.join(class_0_path,os.path.basename(file)))
    
    for index, file in enumerate(files_1):
        shutil.copyfile(file, os.path.join(class_1_path,os.path.basename(file)))

In [ ]:
create_data_repos("../../train_data",part_files_0_train, part_files_1_train )

In [ ]:
create_data_repos("../../test_data",part_files_0_test, part_files_1_test )

# Split trainset to labeled trainset and unlabeled trainset

We will take a small portion of the data as labeled training data and then a large portion as unlabeled.

In [ ]:
result = extract_train_test_data("../../train_data", 20)
part_files_0_train, part_files_1_train, part_files_0_sub_train, part_files_1_sub_train = result

Creation of labelled data

In [ ]:
create_data_repos("../../sub_train_data",part_files_0_sub_train, part_files_1_sub_train)

In [ ]:
def create_unlabeled_repos(output_folder,files_0, files_1):
    os.makedirs(output_folder,exist_ok=True)
    all_files = files_0 + files_1
    random.shuffle(all_files)
    for file in all_files:
        shutil.copyfile(file, os.path.join(output_folder,os.path.basename(file)))

In [ ]:
create_unlabeled_repos("../../unlabeled_data/data",part_files_0_train, part_files_1_train)

In [24]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    featurewise_center=True, 
    featurewise_std_normalization=True,
    zca_whitening=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2) # set validation split

labeled_subset = train_datagen.flow_from_directory(
                                "../../sub_train_data/",
                                target_size=(IMAGE_RESIZE, IMAGE_RESIZE),
                                batch_size=BATCH_SIZE_TRAINING_LABELED_SUBSET,
                                class_mode='categorical',
                                subset='training') # set as training data

"""
validation_generator = train_datagen.flow_from_directory(
                                "train/", # same directory as training data
                                target_size=(IMAGE_RESIZE, IMAGE_RESIZE),
                                batch_size=BATCH_SIZE_VALIDATION,
                                class_mode='categorical',
                                subset='validation') # set as validation data
"""

Found 35523 images belonging to 2 classes.


'\nvalidation_generator = train_datagen.flow_from_directory(\n                                "train/", # same directory as training data\n                                target_size=(IMAGE_RESIZE, IMAGE_RESIZE),\n                                batch_size=BATCH_SIZE_VALIDATION,\n                                class_mode=\'categorical\',\n                                subset=\'validation\') # set as validation data\n'

In [27]:
unlabeled_dataset = train_datagen.flow_from_directory(
                                "../../unlabeled_data/",
                                target_size=(IMAGE_RESIZE, IMAGE_RESIZE),
                                batch_size=BATCH_SIZE_TRAINING_UNLABELED_SUBSET,
                                class_mode=None,
                                subset='training') # set as training data


Found 142094 images belonging to 1 classes.


In [5]:
x_batch, y_batch = next(labeled_subset)

C:\ProgramData\Anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\ProgramData\Anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:735: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


In [19]:
print(len(y_batch))

16


In [ ]:
for i in range (0,16):
    image = x_batch[i]
    #plt.imshow(image.transpose(2,1,0))
    plt.imshow(image)
    plt.show()

In [ ]:
x_batch[0].shape

# Create generator algo

In [6]:
# Generator
# Reference: Chapter 20, GANs in Python by Jason Brownlee, Chapter 7 of GANs in Action
def generator_network(latent_dim):
    in_lat = Input(shape=(latent_dim,))

    n_nodes = 256 * 3 * 3
    gen = Dense(n_nodes)(in_lat)
    gen = Reshape((3, 3, 256))(gen)

    gen = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same')(gen)
    # gen = BatchNormalization()(gen)
    gen = LeakyReLU(alpha=0.01)(gen)

    gen = Conv2DTranspose(64, (3,3), strides=(2,2), padding='same')(gen)
    # gen = BatchNormalization()(gen)
    gen = LeakyReLU(alpha=0.01)(gen)
    
    gen = Conv2DTranspose(32, (3,3), strides=(2,2), padding='same')(gen)
    # gen = BatchNormalization()(gen)
    gen = LeakyReLU(alpha=0.01)(gen)
    
    gen = Conv2DTranspose(16, (3,3), strides=(1,1), padding='same')(gen)
    # gen = BatchNormalization()(gen)
    gen = LeakyReLU(alpha=0.01)(gen)
    
    gen = Conv2DTranspose(8, (3,3), strides=(2,2), padding='same')(gen)
    # gen = BatchNormalization()(gen)
    gen = LeakyReLU(alpha=0.01)(gen)
    
    gen = ZeroPadding2D()(gen)
    
    out_layer = Conv2DTranspose(3, (3,3), strides=(1, 1), activation='tanh', padding='same')(gen)

    model = Model(in_lat, out_layer)

    return model

# Create Discriminator

In [7]:
# Discriminator: Shared weights between the supervised classifier and standard discriminator 
# as found in GANs
# Reference: Chapter 20, GANs in Python by Jason Brownlee, Chapter 7 of GANs in Action

# Activation function for the discriminator as proposed in https://arxiv.org/abs/1606.03498
def custom_activation(output):
    logexpsum = K.sum(K.exp(output), axis=-1, keepdims=True)
    result = logexpsum / (logexpsum + 1.0)
    return result

def disc_network(in_shape=(50,50,3), n_classes=2):
    in_image = Input(shape=in_shape)
 
    fe = Conv2D(32, (3,3), strides=(2,2), padding='same')(in_image)
    fe = LeakyReLU(alpha=0.01)(fe)

    fe = Conv2D(64, (3,3), strides=(2,2), padding='same')(fe)
    fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.01)(fe)
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
    fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.01)(fe)

    fe = Dropout(0.4)(fe)
    fe = Flatten()(fe)

    fe = Dense(n_classes)(fe)

    # Supervised classifier
    c_out_layer = Activation('softmax')(fe)
    c_model = Model(in_image, c_out_layer)

    # Traditional discriminator as found in GANs
    d_out_layer = Lambda(custom_activation)(fe)
    d_model = Model(in_image, d_out_layer)

    return d_model, c_model

In [8]:
# Generator network
generator = generator_network(latent_dim=128)
generator.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
dense (Dense)                (None, 2304)              297216    
_________________________________________________________________
reshape (Reshape)            (None, 3, 3, 256)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 6, 6, 128)         295040    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 6, 6, 128)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 12, 12, 64)        73792     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 12, 12, 64)        0     

In [9]:
# Discriminator and classifier
disc_network, classifier = disc_network()
print(disc_network.summary())
print(classifier.summary())

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 50, 50, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 25, 25, 32)        896       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
batch_normalization (BatchNo (None, 13, 13, 64)        256       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 128)         7385

In [11]:
# Discriminator and classifier
disc_network, classifier = disc_network()
print(disc_network.summary())
print(classifier.summary())

TypeError: __call__() missing 1 required positional argument: 'inputs'

### Run forward passes

In [12]:
#x_batch[0].shape
np.expand_dims(x_batch[0], 0).shape

(1, 50, 50, 3)

In [13]:
# Run forward passes
#x_batch[0].shape
#print(classifier(np.expand_dims(X_train_sub[0], 0)).shape)
print(classifier(np.expand_dims(x_batch[0], 0)).shape)
print(disc_network(np.expand_dims(x_batch[0], 0)).shape)
print(generator(np.random.normal(size=(1, 128))).shape)

(1, 2)
(1, 1)

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

(1, 50, 50, 3)


### Define the optimizers

In [14]:
c_optimizer = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
d_optimizer = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
g_optimizer = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)

## Define the loss functions

In [37]:
#classifier_loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
classifier_loss_fn = tf.keras.losses.CategoricalCrossentropy()
disc_loss_fn = tf.keras.losses.BinaryCrossentropy()
gan_loss_fn = tf.keras.losses.BinaryCrossentropy()

The shared weights of the classifier and the discriminator would be updated on a set of 32 images:

- 16 images from the set of only hundred labeled examples.
- 8 images from the unlabeled examples.
- 8 fake images generated by the generator.

In [38]:
@tf.function
def train_classifier(images, labels, classifier, classifier_loss_fn):
    with tf.GradientTape() as tape:
        logits = classifier(images)
        classification_loss = classifier_loss_fn(labels, logits)
        
    
    grads = tape.gradient(classification_loss, classifier.trainable_weights)
    
    return classification_loss, grads

@tf.function
def train_disc(images, labels, disc_network, disc_loss_fn):
    with tf.GradientTape() as tape:
        logits = disc_network(images)
        disc_loss = disc_loss_fn(labels, logits)
    
    grads = tape.gradient(disc_loss, disc_network.trainable_weights)
    
    return disc_loss, grads

@tf.function
def train_gen(random_latent_vectors, labels, disc_network, generator, gan_loss_fn):
    with tf.GradientTape() as tape:
        logits = disc_network(generator(random_latent_vectors))
        g_loss = gan_loss_fn(labels, logits)

    grads = tape.gradient(g_loss, generator.trainable_weights)
    
    return g_loss, grads

In [39]:
################## Training ##################
##############################################
for epoch in tqdm(range(7500)):
    # Define objects to calculate the mean losses across each epoch
    c_loss_mean = tf.keras.metrics.Mean()
    d_loss_mean = tf.keras.metrics.Mean()
    g_loss_mean = tf.keras.metrics.Mean()
    
    # Train the classifier
    # for (images, labels) in labeled_subset:
    images, labels = next(iter(labeled_subset)) # 16 images
    classification_loss, grads = train_classifier(images, labels, classifier, classifier_loss_fn)
    c_optimizer.apply_gradients(zip(grads, classifier.trainable_weights)) # The shared weights of the classifier and the discriminator will therefore be updated on a total of 32 images
    c_loss_mean.update_state(classification_loss)
        
    ## Train discriminator and generator ##
    #######################################

    # Train discriminator
    real_images = next(iter(unlabeled_dataset)) # 8 real images
    batch_size = tf.shape(real_images)[0]
    random_latent_vectors = tf.random.normal(shape=(batch_size, 128))
    
    generated_images = generator(random_latent_vectors) # 8 fake images
    combined_images = tf.concat([generated_images, real_images], axis=0) # 16 total images
    combined_labels = tf.concat(
        [tf.zeros((batch_size, 1)), tf.ones((batch_size, 1))], axis=0
    ) # 0 -> Fake images, 1 -> Real images

    disc_loss, grads = train_disc(combined_images, combined_labels, disc_network, disc_loss_fn)
    d_optimizer.apply_gradients(zip(grads, disc_network.trainable_weights)) # The shared weights of the classifier and the discriminator will therefore be updated on a total of 32 images
    d_loss_mean.update_state(disc_loss)

    # Train the generator via signals from the discriminator
    random_latent_vectors = tf.random.normal(shape=(batch_size*4, 128)) # 32 images
    misleading_labels = tf.ones((batch_size*4, 1))

    g_loss, grads = train_gen(random_latent_vectors, misleading_labels, disc_network, generator, gan_loss_fn)
    g_optimizer.apply_gradients(zip(grads, generator.trainable_weights))
    g_loss_mean.update_state(g_loss)
        
    if epoch % 100 == 0:
        print("epoch: {} classification loss: {:.3f} dicriminator loss: {:.3f} gan loss:{:.3f}".format(
            epoch,
            c_loss_mean.result(),
            d_loss_mean.result(),
            g_loss_mean.result()
        ))

  0%|          | 0/7500 [00:00<?, ?it/s]

16
logits : 16
labels : 16


C:\ProgramData\Anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\ProgramData\Anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:735: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
  0%|          | 1/7500 [00:02<4:13:13,  2.03s/it]

epoch: 0 classification loss: 0.725 dicriminator loss: 0.756 gan loss:0.406
16

C:\ProgramData\Anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\ProgramData\Anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:735: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
  0%|          | 2/7500 [00:02<3:11:25,  1.53s/it]

...

100%|█████████▉| 7499/7500 [34:44<00:00,  4.73it/s]

16


100%|██████████| 7500/7500 [34:44<00:00,  4.65it/s]

16


100%|██████████| 7500/7500 [34:44<00:00,  3.60it/s]


In [40]:
test_training_datagen = ImageDataGenerator(rescale=1./255) 
test_training_generator = test_training_datagen.flow_from_directory(
                                "../../train_data",
                                target_size=(IMAGE_RESIZE, IMAGE_RESIZE),
                                batch_size=BATCH_SIZE_TRAINING_LABELED_SUBSET,
                                class_mode='categorical') # set as training data
steps = len(test_training_generator.filenames)/BATCH_SIZE_TRAINING_LABELED_SUBSET
result = classifier.predict_generator(test_training_generator,verbose=1,steps=steps)

Found 222020 images belonging to 2 classes.
13877/13876 [==============================] - 3143s 227ms/step


In [ ]:
acc

In [41]:
result

array([[0.84032816, 0.15967178],
       [0.53956366, 0.46043634],
       [0.95922875, 0.04077128],
       ...,
       [0.10183576, 0.8981643 ],
       [0.974668  , 0.02533193],
       [0.88798755, 0.11201245]], dtype=float32)

In [43]:
classifier.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

In [44]:
loss, acc = classifier.evaluate_generator(test_training_generator, steps=steps, verbose=0)

In [45]:
acc

0.8404423

In [46]:
test_testing_datagen = ImageDataGenerator(rescale=1./255) 
test_testing_generator = test_testing_datagen.flow_from_directory(
                                "../../test_data",
                                target_size=(IMAGE_RESIZE, IMAGE_RESIZE),
                                batch_size=BATCH_SIZE_TRAINING_LABELED_SUBSET,
                                class_mode='categorical') # set as training data
steps = len(test_testing_generator.filenames)/BATCH_SIZE_TRAINING_LABELED_SUBSET
#result = classifier.predict_generator(test_training_generator,verbose=1,steps=steps)
loss, acc = classifier.evaluate_generator(test_testing_generator, steps=steps, verbose=0)

Found 55504 images belonging to 2 classes.


In [47]:
acc

0.7418204

In [ ]:
# Performance on the entire training set
accuracy_metric = tf.keras.metrics.CategoricalAccuracy()
accuracy_metric.update_state(y_train, classifier.predict(X_train))
print("Accuracy for the full training set {:.3f}%".format(accuracy_metric.result().numpy()*100))

In [49]:
latent = np.random.normal(size=(1, 128))
image = generator.predict(latent)
#plt.imshow(image.transpose(2,1,0))
plt.imshow(image)
plt.show()

ValueError: Error when checking input: expected input_1 to have 2 dimensions, but got array with shape (1, 1, 128)